In [108]:
import pandas as pd
from dateutil.relativedelta import relativedelta

df = pd.read_csv('https://www.dropbox.com/s/832hth1es40f4g7/newfutures.csv?dl=1', index_col=0)
df = df[df.date>='2010-01-01']
df.date = pd.to_datetime(df.date)

# Function to add months and set day to 15th
def add_months_and_set_day(date, months_to_add):
    new_date = date + relativedelta(months=int(months_to_add))
    new_date = new_date.replace(day=15)
    return new_date

# Apply the function to create the new column
df['new_to_date'] = df.apply(
    lambda row: add_months_and_set_day(row['date'], row['months past front month']), 
    axis=1
)
df = df.reset_index(drop=True).set_index(['code', 'date', 'new_to_date']).settle

In [112]:
df.index.unique('date')

DatetimeIndex(['2010-01-14', '2010-01-28', '2010-02-11', '2010-02-25',
               '2010-03-11', '2010-03-25', '2010-04-08', '2010-04-22',
               '2010-05-06', '2010-05-20',
               ...
               '2022-05-18', '2022-06-01', '2022-06-15', '2022-06-29',
               '2014-02-23', '2012-01-16', '2017-01-16', '2017-07-04',
               '2017-01-02', '2020-09-07'],
              dtype='datetime64[ns]', name='date', length=324, freq=None)

In [109]:
yen = df.loc[('CME_JY', '2022-06-01')]
peso = df.loc[('CME_MP', '2022-06-01')]
gold = df.loc[('CME_GC', '2022-06-01')]
sp500 = df.loc[('CME_ES', '2020-06-01')]
crude1 = df.loc[('CME_CL', '2020-04-20')]
crude2 = df.loc[('CME_CL', '2022-03-08')]
gas = df.loc[('CME_NG', '2018-06-08')]
corn = df.loc[('CME_C', '2018-06-08')]

In [111]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with 2x2 subplots with horizontal spacing
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Gold (2022-06-01)', 'S&P 500 (2020-06-01)', 
                    'Japanese Yen (2022-06-01)', 'Mexican Peso (2022-06-01)'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]],
    horizontal_spacing=0.12  # Add horizontal spacing between columns
)

# Add Gold trace (top left)
fig.add_trace(
    go.Scatter(x=gold.index, y=gold.values, 
               name='Gold',
               mode='lines+markers',
               hovertemplate='%{x|%b %Y}<br>Price: %{y}<extra></extra>'),
    row=1, col=1
)

# Add S&P 500 trace (top right)
fig.add_trace(
    go.Scatter(x=sp500.index, y=sp500.values, 
               name='S&P 500',
               mode='lines+markers',
               hovertemplate='%{x|%b %Y}<br>Price: %{y}<extra></extra>'),
    row=1, col=2
)

# Add Yen trace (bottom left)
fig.add_trace(
    go.Scatter(x=yen.index, y=yen.values, 
               name='Japanese Yen',
               mode='lines+markers',
               hovertemplate='%{x|%b %Y}<br>Price: %{y}<extra></extra>'),
    row=2, col=1
)

# Add Peso trace (bottom right)
fig.add_trace(
    go.Scatter(x=peso.index, y=peso.values, 
               name='Mexican Peso',
               mode='lines+markers',
               hovertemplate='%{x|%b %Y}<br>Price: %{y}<extra></extra>'),
    row=2, col=2
)

# Update layout - set width=None for auto-sizing
fig.update_layout(
    height=600,
    width=None,  # Auto-adjust width to container
    showlegend=False,
    autosize=True,  # Enable responsive sizing
    template='plotly_white',  # Use plotly white template
    # Make subplot titles smaller
    annotations=[dict(font=dict(size=12)) for _ in fig['layout']['annotations']]
)

# Update axes labels with smaller font
fig.update_xaxes(title_text="Contract Date", row=2, col=1, title_font=dict(size=11), tickfont=dict(size=9))
fig.update_xaxes(title_text="Contract Date", row=2, col=2, title_font=dict(size=11), tickfont=dict(size=9))
fig.update_yaxes(title_text="Futures Price", row=1, col=1, title_font=dict(size=11), tickfont=dict(size=9))
fig.update_yaxes(title_text="", row=1, col=2, title_font=dict(size=10), tickfont=dict(size=9))
fig.update_yaxes(title_text="Futures Price", row=2, col=1, title_font=dict(size=11), tickfont=dict(size=9))
fig.update_yaxes(title_text="", row=2, col=2, title_font=dict(size=10), tickfont=dict(size=9))

# Update all tick labels to be smaller
fig.update_xaxes(tickangle=45, tickfont=dict(size=9))
fig.update_yaxes(tickfont=dict(size=9))

# Update subplot titles font size
for annotation in fig['layout']['annotations']:
    annotation['font'] = dict(size=12)

fig.show()